In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from tabulate import tabulate
import statsmodels
import math
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

In [5]:
# Reading data from the csv
df = pd.read_csv(r"..\Excels\merged_table.csv")
df.columns
dummy_years = pd.get_dummies(df['year'], prefix='year')
gwq='calcium'
# Concatenate the original DataFrame with the dummy variable DataFrame
df = pd.concat([df, dummy_years], axis=1)

# Convert numeric columns to appropriate data type
df['SDP'] = pd.to_numeric(df['SDP'], errors='coerce')
df[gwq] = pd.to_numeric(df[gwq], errors='coerce')
df['Gini'] = pd.to_numeric(df['Gini'], errors='coerce')

# Drop rows with NaN values
df.dropna(subset=['SDP', gwq,'Gini'], inplace=True)

x = df['SDP'].values.reshape(-1, 1)  # Independent variable
y = df[gwq].values  # Dependent variable
x2 = x ** 2
x3 = x ** 3
x4 = df['Gini'].values.reshape(-1, 1)  # Fourth variable

# Create polynomial features
poly = PolynomialFeatures(degree=3)  # You can change the degree here
X_poly = poly.fit_transform(x)

# Add fourth variable
X_poly = np.concatenate((X_poly, x4), axis=1)

final_x = X_poly.copy()
years = ['year_2000', 'year_2001', 'year_2002', 'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017']

for yr in years:
    curr_year = np.array(df[yr]).reshape(-1, 1)  # Reshape to make it a column vector
    new_x_poly = []
    
    # Multiply each element of X_poly by the current year and append to new_x_poly
    for i in range(len(X_poly)):
        l = []
        for j in range(len(X_poly[i])):
            l.append(X_poly[i][j] * curr_year[i])
        new_x_poly.append(l)
        
    # Convert new_x_poly to numpy array
    new_x_poly = np.array(new_x_poly).reshape(-1, len(X_poly[0]))  # Reshape to match final_x
    
    # Add new_x_poly to final_x by horizontally stacking them
    final_x = np.hstack((final_x, new_x_poly))
    
    # print(final_x.shape)



In [6]:

y = df[gwq].values  # Dependent variable

dummy_year_model = LinearRegression(fit_intercept=True).fit(final_x, y)

# Get coefficients and intercept
intercept = dummy_year_model.intercept_
coefficients = dummy_year_model.coef_

# Print coefficients
print("Intercept:", intercept)
print("Coefficients:", coefficients)

# Evaluate R^2
r_squared = dummy_year_model.score(final_x, y)
print("R^2:", r_squared)

Intercept: 20.2569417446771
Coefficients: [-1.24566708e-08  2.68192462e-04 -4.70576851e-10  2.45960888e-16
  3.91266543e-08 -7.82222660e-09  8.07520043e-04 -6.15941384e-09
  1.15342836e-14  1.39817822e-07  1.52751189e-07  6.24165941e-04
 -4.27064978e-09  6.66420564e-15  1.09810265e-07 -7.18997540e-09
  8.56888689e-04 -5.89584828e-09  9.67917857e-15  1.81256362e-07
  7.94467894e-08  2.43263559e-03 -2.16615540e-08  4.21130695e-14
  2.41557262e-08  1.19742099e-08  8.23771576e-04 -6.73819106e-09
  1.24024175e-14  3.33566320e-09  4.30393535e-10  5.52793376e-04
 -3.30948858e-09  4.77718451e-15  3.05527551e-10 -2.85362505e-09
  3.51130482e-04 -1.24737117e-09  1.07951300e-15 -4.50128529e-10
 -2.99817775e-09  4.01255554e-04 -1.93446576e-09  2.21827764e-15
 -5.41594230e-10 -1.29163432e-09  4.26381045e-04 -2.08336980e-09
  2.34567140e-15 -4.29071537e-11 -5.80215680e-09  2.06333346e-04
 -9.63405915e-10  1.06425285e-15 -1.39353782e-09 -5.36951244e-09
 -9.94127701e-06  1.16057085e-09 -3.25889489e-15